# Threshold images

K.Garner, 2020

This wf takes an spmT image, anatomically masks it. The remaining is saved.
Designed to be repeated over t's and over masks.

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

210503-16:30:05,358 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="thrsh") # workflow to run the analysis

## Data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['ts','msks']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']],
                           'msks': [['sub', 'TR']]}
dg.inputs.field_template = {'msks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/MASKS/*.nii.gz',
                            'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-%s/TR%s/FLGLM/tSNR_beta*.nii.gz'}

In [4]:
# # debugging
dg.inputs.sub = '01'
dg.inputs.TR = '700ECHO1'
res = dg.run()
res.outputs
ncon = len(res.outputs.ts)
ncon

210503-16:30:07,630 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmp_v1ldwj1/data-grabber".
210503-16:30:07,635 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210503-16:30:07,641 nipype.workflow INFO:
	 [Node] Finished "data-grabber".


6

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
#printSubPath(res.outputs.msks)

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/"
substitutions = [('_TR_([0-9]*)ECHO([0-9]*)_sub_([0-9]*)', ''),
                 ('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_thrsh-t([0-9]*)', ''),
                 ('_absthat([0-9]*)', ''),
                 ('_mask-t([0-9])*', ''),
                 ('_multi([0-9])*', '')] 
ds.inputs.regexp_substitutions = substitutions

In [8]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920','700ECHO1', '700ECHO2'])]
#datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700ECHO1', '700ECHO2'])]
#datinf.iterables = [('sub', ['05']), ('TR', ['700'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']),
 ('TR', ['700', '1510', '1920', '700ECHO1', '700ECHO2'])]

## Gzip interface

get .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

## functions for list cats

In [10]:
def tcat(fullFileList, nMasks=9):
    return fullFileList*nMasks

In [11]:
def maskcat(fullFileList, nCon=6):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

## define output names for t x msk

In [12]:
#tnms = ['abs_spmT_0001', 'abs_spmT_0002', 'abs_spmT_0005', 'abs_spmT_0007', 'abs_spmT_0013', 'abs_spmT_0014', 'abs_spmT_0015']
tnms = ['tSNR_beta_0001', 'tSNR_beta_0002', 'tSNR_beta_0003', 'tSNR_beta_0004', 'tSNR_beta_0013', 'tSNR_beta_0014']
msks = ['rCN', 'rFEF', 'rGPe', 'rGPi', 'rIPS', 'rLOC', 'rPut', 'rSTN', 'rVS']
touts = ['th'+i+'_'+n+'.nii.gz' for i in msks for n in tnms]

In [13]:
touts

['thrCN_tSNR_beta_0001.nii.gz',
 'thrCN_tSNR_beta_0002.nii.gz',
 'thrCN_tSNR_beta_0003.nii.gz',
 'thrCN_tSNR_beta_0004.nii.gz',
 'thrCN_tSNR_beta_0013.nii.gz',
 'thrCN_tSNR_beta_0014.nii.gz',
 'thrFEF_tSNR_beta_0001.nii.gz',
 'thrFEF_tSNR_beta_0002.nii.gz',
 'thrFEF_tSNR_beta_0003.nii.gz',
 'thrFEF_tSNR_beta_0004.nii.gz',
 'thrFEF_tSNR_beta_0013.nii.gz',
 'thrFEF_tSNR_beta_0014.nii.gz',
 'thrGPe_tSNR_beta_0001.nii.gz',
 'thrGPe_tSNR_beta_0002.nii.gz',
 'thrGPe_tSNR_beta_0003.nii.gz',
 'thrGPe_tSNR_beta_0004.nii.gz',
 'thrGPe_tSNR_beta_0013.nii.gz',
 'thrGPe_tSNR_beta_0014.nii.gz',
 'thrGPi_tSNR_beta_0001.nii.gz',
 'thrGPi_tSNR_beta_0002.nii.gz',
 'thrGPi_tSNR_beta_0003.nii.gz',
 'thrGPi_tSNR_beta_0004.nii.gz',
 'thrGPi_tSNR_beta_0013.nii.gz',
 'thrGPi_tSNR_beta_0014.nii.gz',
 'thrIPS_tSNR_beta_0001.nii.gz',
 'thrIPS_tSNR_beta_0002.nii.gz',
 'thrIPS_tSNR_beta_0003.nii.gz',
 'thrIPS_tSNR_beta_0004.nii.gz',
 'thrIPS_tSNR_beta_0013.nii.gz',
 'thrIPS_tSNR_beta_0014.nii.gz',
 'thrLOC_tSNR_be

## FSL functionality to apply anatomical mask

In [14]:
# maskt = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-t')
# maskt.inputs.out_file = touts
# maskt.inputs.nan2zeros = True

maskt = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'operand_file', 'out_file'], name='mask-t')
maskt.inputs.operation = 'mul'
maskt.inputs.nan2zeros = True
maskt.inputs.out_file = touts

## to take the absolute value of the image

In [15]:
# absv = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file', 'out_file'], name='absthat')
# absv.inputs.operation = 'abs'
# absv.out_file = touts

# rounding test

In [16]:
# multIm = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'out_file'], name='multi')
# multIm.inputs.operand_value = 10
# multIm.inputs.operation = 'mul'
# multIm.inputs.out_file = touts

## and to apply threshold

In [17]:
# thrsht = pe.MapNode(fsl.Threshold(), iterfield = ['in_file', 'out_file'], name='thrsh-t')
# thrsht.inputs.thresh = 2.326 # p=.01 - 2 tailed
# thrsht.inputs.out_file = touts
#thrsht.inputs.internal_datatype = 'input'
# thrsht.inputs.use_robust_range = True
# thrsht.inputs.use_nonzero_voxels = True

## Connect workflow

In [18]:
# thrsh.connect([(datinf, dg, [('sub', 'sub')]),
#               (datinf, dg, [('TR', 'TR')]),
#               (dg, ds, [(('msks', printSubPath), 'container')]),
# #               (dg, gzip, [('ts', 'in_file')]), # zip the t-images # use these 2 lines if .nii -> .nii.gz is required
# #               (gzip, maskt, [(('out_file', tcat), 'in_file')]),
#               (dg, maskt, [(('ts', tcat), 'in_file')]), 
#               (dg, maskt, [(('msks', maskcat), 'mask_file')]),
#               (maskt, absv, [('out_file', 'in_file')]), 
#               (absv, thrsht, [('out_file', 'in_file')]),
#               (thrsht, ds, [('out_file', 'THRSH')])
# ])
# res = thrsh.run()

## Debugging workflow

In [19]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('msks', printSubPath), 'container')]),
               (dg, maskt, [(('ts', tcat), 'in_file')]), 
               (dg, maskt, [(('msks', maskcat), 'operand_file')]),
               (maskt, ds, [('out_file', 'THRSH')])
])
res = thrsh.run()

210503-16:30:07,762 nipype.workflow INFO:
	 Workflow thrsh settings: ['check', 'execution', 'logging', 'monitoring']
210503-16:30:07,873 nipype.workflow INFO:
	 Running serially.
210503-16:30:07,874 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.data-grabber" in "/tmp/tmp_v1ldwj1/data-grabber".
210503-16:30:07,879 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210503-16:30:07,885 nipype.workflow INFO:
	 [Node] Finished "thrsh.data-grabber".
210503-16:30:07,885 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.mask-t" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t".
210503-16:30:07,907 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t0".
210503-16:30:07,911 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO

210503-16:30:10,143 nipype.workflow INFO:
	 [Node] Finished "_mask-t12".
210503-16:30:10,146 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t13" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t13".
210503-16:30:10,150 nipype.workflow INFO:
	 [Node] Running "_mask-t13" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/FLGLM/tSNR_beta_0002.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700ECHO2/MASKS/rGPe_trans.nii.gz thrGPe_tSNR_beta_0002.nii.gz
210503-16:30:10,314 nipype.workflow INFO:
	 [Node] Finished "_mask-t13".
210503-16:30:10,316 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t14" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t14".
210503-16:30:10,324 nipype.workflow INFO:
	 [Node] Running "_mask-t14" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratc

210503-16:30:12,614 nipype.workflow INFO:
	 [Node] Finished "_mask-t26".
210503-16:30:12,616 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t27" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t27".
210503-16:30:12,621 nipype.workflow INFO:
	 [Node] Running "_mask-t27" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/FLGLM/tSNR_beta_0004.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700ECHO2/MASKS/rIPS_trans.nii.gz thrIPS_tSNR_beta_0004.nii.gz
210503-16:30:12,799 nipype.workflow INFO:
	 [Node] Finished "_mask-t27".
210503-16:30:12,801 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t28" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t28".
210503-16:30:12,806 nipype.workflow INFO:
	 [Node] Running "_mask-t28" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratc

210503-16:30:15,60 nipype.workflow INFO:
	 [Node] Finished "_mask-t40".
210503-16:30:15,62 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t41" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t41".
210503-16:30:15,67 nipype.workflow INFO:
	 [Node] Running "_mask-t41" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/FLGLM/tSNR_beta_0014.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700ECHO2/MASKS/rPut_trans.nii.gz thrPut_tSNR_beta_0014.nii.gz
210503-16:30:15,241 nipype.workflow INFO:
	 [Node] Finished "_mask-t41".
210503-16:30:15,243 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t42" in "/tmp/tmpfoc_pqyp/thrsh/_TR_700ECHO2_sub_05/mask-t/mapflow/_mask-t42".
210503-16:30:15,248 nipype.workflow INFO:
	 [Node] Running "_mask-t42" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/q

210503-16:30:17,349 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH/_TR_700ECHO2_sub_05/_mask-t0/thrCN_tSNR_beta_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH///thrCN_tSNR_beta_0001.nii.gz
210503-16:30:17,351 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH/_TR_700ECHO2_sub_05/_mask-t1/thrCN_tSNR_beta_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH///thrCN_tSNR_beta_0002.nii.gz
210503-16:30:17,353 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH/_TR_700ECHO2_sub_05/_mask-t2/thrCN_tSNR_beta_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRSH///thrCN_tSNR_beta_0003.nii.gz
210503-16:30:17,355 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700ECHO2/THRS

...

210503-16:35:03,86 nipype.workflow INFO:
	 [Node] Finished "_mask-t14".
210503-16:35:03,88 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t15" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t15".
210503-16:35:03,92 nipype.workflow INFO:
	 [Node] Running "_mask-t15" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/FLGLM/tSNR_beta_0004.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rGPe_trans.nii.gz thrGPe_tSNR_beta_0004.nii.gz
210503-16:35:03,262 nipype.workflow INFO:
	 [Node] Finished "_mask-t15".
210503-16:35:03,264 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t16" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t16".
210503-16:35:03,269 nipype.workflow INFO:
	 [Node] Running "_mask-t16" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/

210503-16:35:05,761 nipype.workflow INFO:
	 [Node] Finished "_mask-t28".
210503-16:35:05,763 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t29" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t29".
210503-16:35:05,768 nipype.workflow INFO:
	 [Node] Running "_mask-t29" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/FLGLM/tSNR_beta_0014.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rIPS_trans.nii.gz thrIPS_tSNR_beta_0014.nii.gz
210503-16:35:05,975 nipype.workflow INFO:
	 [Node] Finished "_mask-t29".
210503-16:35:05,978 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t30" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t30".
210503-16:35:05,982 nipype.workflow INFO:
	 [Node] Running "_mask-t30" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIW

210503-16:35:08,436 nipype.workflow INFO:
	 [Node] Finished "_mask-t42".
210503-16:35:08,438 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t43" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t43".
210503-16:35:08,443 nipype.workflow INFO:
	 [Node] Running "_mask-t43" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/FLGLM/tSNR_beta_0002.nii.gz -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/MASKS/rSTN_trans.nii.gz thrSTN_tSNR_beta_0002.nii.gz
210503-16:35:08,627 nipype.workflow INFO:
	 [Node] Finished "_mask-t43".
210503-16:35:08,629 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t44" in "/tmp/tmprh_uq8ts/thrsh/_TR_700_sub_01/mask-t/mapflow/_mask-t44".
210503-16:35:08,634 nipype.workflow INFO:
	 [Node] Running "_mask-t44" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIW

210503-16:35:10,625 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t5/thrCN_tSNR_beta_0014.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrCN_tSNR_beta_0014.nii.gz
210503-16:35:10,626 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t6/thrFEF_tSNR_beta_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrFEF_tSNR_beta_0001.nii.gz
210503-16:35:10,628 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t7/thrFEF_tSNR_beta_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrFEF_tSNR_beta_0002.nii.gz
210503-16:35:10,630 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t8/thrFEF_tSNR_beta_000

210503-16:35:10,676 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t35/thrLOC_tSNR_beta_0014.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrLOC_tSNR_beta_0014.nii.gz
210503-16:35:10,677 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t36/thrPut_tSNR_beta_0001.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrPut_tSNR_beta_0001.nii.gz
210503-16:35:10,679 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t37/thrPut_tSNR_beta_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH///thrPut_tSNR_beta_0002.nii.gz
210503-16:35:10,681 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR700/THRSH/_TR_700_sub_01/_mask-t38/thrPut_tSNR_be